In [ ]:
import time
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
data_dict = {'name':[], 'release date':[], 'developer': [], 'genre':[], 'rating':[], 'user rating':[],'age':[],'platforms':[],'number of players':[],'publisher':[]} # Data Structure


In [ ]:
def scrape_each_game(url):#will scrape information from each game page
 
    
    
    #credit for soulution:https://towardsdatascience.com/web-scraping-metacritic-reviews-using-beautifulsoup-63801bbe200e
    #note:couldnt pass 403 error
    
    userAgent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=userAgent)
    soup = BeautifulSoup(response.text,"html.parser")
    #print('-------------------------------------------------------------')
    
    #name################################################################################
    
    if soup.find('div', class_='product_title') is None:
        return
        
    #print(name)
    else: 
        name = soup.find('div', class_='product_title').find('h1').get_text()
        data_dict['name'].append(name)
       
    
    
    #platform#############################################################################
    if soup.find('span', class_='platform') is None:
        data_dict['platforms'].append(np.nan)
    else:
        platform = soup.find('span', class_='platform').get_text().strip()
        #print(platform)
        data_dict['platforms'].append(platform)
    
    
    
    #release date#########################################################################
    if soup.find('li',{"class":"summary_detail release_data"}).find(class_='data') is None:
        data_dic['release date'].append(np.nan)
    else:
        release_date = soup.find('li',{"class":"summary_detail release_data"}).find(class_='data').get_text().strip()
        #print(release_date)
        data_dict['release date'].append(release_date)
    
    
    
    #developer###########################################################################
    developer = soup.find('li', class_='summary_detail developer')
    if developer is not None:
        developer = developer.find('span', class_='data').get_text().strip()
        #print(developer)
        data_dict['developer'].append(developer)
    else:
        developer = np.nan
        #print(developer)
        data_dict['developer'].append(developer)
       
    
    
    #genre###############################################################################    
    genre = []
    for g in soup.find('li', class_='summary_detail product_genre').find_all('span', class_='data'): 
        genre.append(g.get_text().strip())
       
    lst='/'.join(genre)
    #print(lst)
    data_dict['genre'].append(lst)
   


    #rating##############################################################################
    rating = soup.find('div', class_='details main_details').find('span',{'itemprop':'ratingValue'})
    if rating is not None: 
        ratings = rating.get_text().strip()
        data_dict['rating'].append(ratings)
    else:
        rating = np.nan
        #print(rating)
        data_dict['rating'].append(rating)
    
    
    
    #user rating#########################################################################
    user_rating=soup.find(class_='metascore_w user large game positive')
    #note: metacritic devides ratings to colors
    if user_rating is None:
        user_rating=soup.find(class_='metascore_w user large game mixed')
        if user_rating is None:
            user_rating=soup.find(class_='metascore_w user large game negative')
            if user_rating is None:
                user_rating='-'
    if user_rating=='-':
        data_dict['user rating'].append(user_rating)
    else: data_dict['user rating'].append(user_rating.get_text().strip())
    #print(user_rating)
    
    
    
    #age################################################################################
    if soup.find(class_='summary_detail product_rating') is None:
        age='-'
    else:age=soup.find(class_='summary_detail product_rating').find(class_='data').get_text().strip()
    data_dict['age'].append(age)
    #print(age)
    
    
    #number of players####################################################################
    if soup.find(class_='summary_detail product_players') is None:
        num_of_players='-'
    else: num_of_players=soup.find(class_='summary_detail product_players').find(class_='data').get_text().strip()
    data_dict['number of players'].append(num_of_players)
    
    if soup.find('li',{'class':'summary_detail publisher'})  is None:
        data_dict['publisher'].append(np.nan)
    else: 
        publish=soup.find('li',{'class':'summary_detail publisher'}).find('a',href=True).get_text().strip()
        
        data_dict['publisher'].append(publish)
        

In [ ]:
def how_many_pages(response):#will calculate amount of pages to traverse
    soup = BeautifulSoup(response.text, 'html.parser')
    pages = soup.find_all('li', {"class":"page last_page"})
    pagesCleaned = pages[0].find('a', {"class":"page_num"})
    return (pagesCleaned.text)
    
    
    

In [ ]:

#lands on the first page of pc games filtered by highest score of metacritic.
url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page=0'
userAgent = {'User-agent': 'Mozilla/5.0'}
response = requests.get(url, headers=userAgent)
soup = BeautifulSoup(response.text,"html.parser")


amount_of_pages=int(how_many_pages(response))
print("amount of pages:",amount_of_pages)


flag=0
while flag < amount_of_pages: #casting string of how many pages to int.  
   
    for a in soup.find_all('a', {"class":"title"} ,href=True):
        game_url='https://www.metacritic.com'+a['href']#sets the url to review page of the game. 
        scrape_each_game(game_url)
       
    time.sleep(1)    
    flag+=1
    #moves to next page:
        #adds to the url the current page
    url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page='+str(flag)
    userAgent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=userAgent)
    soup = BeautifulSoup(response.text,"html.parser")
    

df=pd.DataFrame(data_dict) 
df.to_csv('dataFrame#1.csv')
display(df)